In [1]:
import sys, os, time, shutil, random
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import omnibelt as belt
import omnifig as fig
import numpy as np
np.set_printoptions(linewidth=120)
# %load_ext autoreload
# %autoreload 2
# %pdb
import pickle
from tabulate import tabulate
from tqdm import tqdm_notebook as tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
# import torchvision.models
from torch.utils.data import Dataset, DataLoader, TensorDataset

%matplotlib notebook
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
# import matplotlib.patches as mpatches
# import matplotlib.pyplot as plt
# import matplotlib.cm
# from matplotlib import animation
# import matplotlib as mpl
# # mpl.rc('image', cmap='gray')
# import seaborn as sns
# import pandas as pd
# from sklearn.preprocessing import normalize

fig.initialize();
# import gpumap
# from c3linearize import linearize, class_graph

In [2]:
import plethora

In [3]:
from plethora.datasets.toy import SwissRollDataset

In [4]:
dataset = SwissRollDataset()

AttributeError: 'BoundDim' object has no attribute 'expanded_len'